In [1]:
import os
from google.cloud import bigquery
from get_bq_data import get_bq_data
from temp_model_trainer import get_predictions
from dotenv import load_dotenv

load_dotenv()

os.environ['GOOGLE_APPLICATION_CREDENTIALS' ] = 'service_keys.json'

client = bigquery. Client()

In [2]:
def aggregate_data():
    """Load dataset, aggregate, and push to Supabase""" 

    existing_data = get_bq_data(client,"state_market") 

    return existing_data

In [3]:
def train_model(df):
    """Load dataset, train model, save predictions to Supabase"""

    # List of features to predict
    features = [
        "median_listing_price",
        "average_listing_price",
        "median_listing_price_per_square_foot",
        "total_listing_count",
        "median_days_on_market"
    ]
    
    for feature in features:
        get_predictions(df, feature)

In [4]:
df = aggregate_data()
df.sort_values(by=['year', 'month', 'state_num'], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,year,month,state_num,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy
0,2017,7,1,198060,-0.004223,0.0654,26519,0.002343,-0.1296,75,...,0.0233,264342,-0.000707,0.0543,28563,-0.004288,-0.1169,0.0789,-0.067376,0.0179
1,2017,7,2,279375,-0.006667,-0.0333,3710,0.092784,-0.0196,53,...,-0.0278,327328,-0.003374,-0.0121,3710,0.092784,-0.0196,0.0000,0.000000,0.0000
2,2017,7,3,312000,-0.018868,0.0576,26807,-0.026015,-0.1148,56,...,0.0086,477781,-0.029825,0.0699,34886,-0.034965,-0.0948,0.2988,-0.048711,0.0294
3,2017,7,4,169900,0.000000,0.0534,15717,-0.013309,-0.1057,73,...,0.0245,235007,0.003810,0.0652,18446,-0.013161,-0.0939,0.1771,0.002831,0.0145
4,2017,7,5,549000,-0.001637,0.0660,71686,0.037319,-0.1431,40,...,-0.0005,1030673,-0.016759,0.1016,106530,0.009907,-0.1262,0.4944,-0.069627,0.0314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5044,2025,9,47,450000,0.000000,0.0228,21543,0.028400,0.2931,45,...,0.0005,639805,0.012100,0.0062,31820,0.022700,0.2011,0.4810,-0.011600,-0.1128
5045,2025,9,48,648900,-0.001600,-0.0016,24463,-0.002500,0.2562,57,...,-0.0035,887686,-0.012300,-0.0109,33936,-0.010900,0.1543,0.3888,-0.011200,-0.1228
5046,2025,9,49,261500,-0.027900,0.0898,4245,0.038700,0.1546,52,...,0.0120,325854,-0.029700,0.0233,6712,0.030100,0.1029,0.5822,-0.010600,-0.0729
5047,2025,9,50,392325,-0.016700,0.0257,13190,0.037400,0.0899,44,...,-0.0036,506718,-0.004000,0.0422,18863,0.000800,0.0766,0.4327,-0.049800,-0.0216


In [5]:
train_model(df)

Initial memory: 1.80 MB
Final memory: 1.20 MB
Reduced by 33.2%
Validation RMSE: 5388.5618, RMSLE: 0.0113, MAE: 4073.1587, MAPE: 0.90%, R²: 0.9985
Validation RMSE: 9823.6407, RMSLE: 0.0203, MAE: 7467.9265, MAPE: 1.61%, R²: 0.9947
Validation RMSE: 8014.6784, RMSLE: 0.0180, MAE: 6254.5021, MAPE: 1.43%, R²: 0.9965
🏃 View run XGB_Darts_Model_median_listing_price at: http://127.0.0.1:5000/#/experiments/1/runs/fd604f5cf55d45e99d4f662350917f21
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023870 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 164580
[LightGBM] [Info] Number of data points in the train set: 3570, number of used features: 662
[LightGBM] [Info] Start training from score 0.638591
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018722 seconds.
You can set `force_col_wise=true` to remove the over

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | LSTM             | 5.1 K  | train
6 | fc              | Sequential       | 51     | train
-------------------------------------------------------------
5.2 K     Trainable params
0         Non-trainable params
5.2 K     Total params
0.021     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

...

🏃 View run LightGBM_Darts_Model_median_days_on_market at: http://127.0.0.1:5000/#/experiments/1/runs/b0672614a78b45e6b4c2e603fe737002
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Validation RMSE: 11.2818, RMSLE: 0.2025, MAE: 9.2040, MAPE: 17.05%, R²: -0.0654
Validation RMSE: 12.4200, RMSLE: 0.2207, MAE: 9.6012, MAPE: 16.43%, R²: -0.2253
Validation RMSE: 13.6370, RMSLE: 0.2367, MAE: 10.5811, MAPE: 17.00%, R²: -0.2500
🏃 View run LSTM_Darts_Model_median_days_on_market at: http://127.0.0.1:5000/#/experiments/1/runs/7bfd431df0644e0792bf3de0da26f953
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
Validation RMSE: 3.4016, RMSLE: 0.0630, MAE: 2.6741, MAPE: 4.98%, R²: 0.9031
Validation RMSE: 5.2041, RMSLE: 0.0900, MAE: 4.1221, MAPE: 7.20%, R²: 0.7849
Validation RMSE: 5.5036, RMSLE: 0.0916, MAE: 3.8735, MAPE: 6.77%, R²: 0.7964
🏃 View run LinearRegression_Darts_Model_median_days_on_market at: http://127.0.0.1:5000/#/experiments/1/runs/154224a019224bee839d1c8d5e737ee3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
Validation RMSE: 3.3795, RMSLE: 0.0558, MAE: 2.3277, MAPE: 4.14%, R²: 0.9044
Validation RMSE: 4.8338, RMSLE: 0.0786, MAE: 3.6961, MAPE: 